In [52]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime
from scipy.optimize import minimize

## Group Assignment
### Team Number: 12
### Team Member Names: Sharuga, Derek, Alex
### Team Strategy Chosen: Market Meet

Disclose any use of AI for this assignment below (detail where and how you used it).  Please see the course outline for acceptable uses of AI.


In [53]:
START_DATE = '2023-11-25'
END_DATE = '2024-11-23'
INVESTMENT = 1000000


In [54]:
# Function to calculate the purchasing fee depending on the number of shares
def calculate_purchase_fee(shares):
    return max(3.95, 0.01*shares)

In [55]:

def validity(tickers):
    final_list = []
    for ticker in tickers:
        #only append the stock to the final stock list if it is valid
        if (not check_delist(ticker) and
            check_currency(ticker) and
            check_trading_days(ticker)):
                final_list.append(ticker)
    return final_list

def check_delist(ticker):
    stock = yf.Ticker(ticker)
    try:
        data = stock.history(period='1d')
        if data.empty:
            #if we can't find any data on the stock, it's delisted
            return True
        else:
            #check that there is actually valid market data for this stock - AI
            if 'Close' not in data.columns or data['Close'].isnull().all():
                #if there are no valid close prices, the stock is delisted
                return True
            return False
    except Exception as e:
        #if there is an error in finding the stock's data, we can assume that it's delisted
        return True

def check_trading_days(ticker):
    min_days=18
    #the stock should have at least 18 trading days (we can check this by checking the last month)
    stock = yf.Ticker(ticker)
    try:
        data = stock.history(period="1mo") #fetches the stock info from the last month
        #AI
        trading_days = data.shape[0]
        return trading_days >= min_days  # Must have at least `min_days` trading days
    except Exception as e:
        return False  # If error, assume it’s invalid

def check_currency(ticker):
    """Check if the stock is in USD or CAD currency."""
    stock = yf.Ticker(ticker)
    try:
        currency = stock.info.get('currency', None)
        return currency in ['USD', 'CAD']
    except Exception as e:
        return False  # If error, assume it’s invalid

all_data = pd.read_csv('Tickers_Example.csv', header=None)

all_data = pd.DataFrame(validity(all_data[0]))
all_data.rename(columns = {0:'Ticker'}, inplace=True)

$AGN: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$CELG: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$MON: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$RTN: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


In [56]:
# Create a list of all the stock betas (retrieved from yfinance) of stock tickers in a list
def get_betas(tickers):
    betas = []
    
    for ticker in tickers:
        betas.append(yf.Ticker(ticker).info.get('beta'))

    return betas

In [57]:
def get_sector(tickers):
    sectors = []

    for ticker in tickers:
        sectors.append(yf.Ticker(ticker).info.get('sector'))

    return sectors

In [79]:
# Add a column containing all the stock betas to [all_data]
all_data.assign(Beta=get_betas(all_data['Ticker']))
all_data.assign(Sector=get_sector(all_data['Ticker']))

,Ticker,Beta,Sector
1,UPS,0.982,Industrials
2,TXN,0.980,Technology
3,USB,1.040,Financial Services
4,UNP,1.056,Industrials
5,BK,1.060,Financial Services
6,BB.TO,1.068,Technology
7,AIG,1.069,Financial Services
8,CAT,1.095,Industrials
9,AMZN,1.146,Consumer Cyclical
10,RY.TO,0.842,Financial Services


In [59]:
# Choose the max(24, data.length) stocks that match the market the closest
def filter_betas(data):
    # Remove any negative betas, as long as data ends with at least 12 characters
    data.sort_values(by='Beta', axis=0, inplace=True, kind='quicksort')
    data.drop(index=data[(data['Beta'] <= 0) & (len(data) > 12)].index, inplace=True)
    data.reset_index(drop=True, inplace=True)

    data['temp'] = abs(1-data['Beta'])
    data.sort_values(by='temp', axis=0, inplace=True, kind='quicksort')    # Sort data by temp
    data.drop(columns='temp', axis=1, inplace=True)    # Remove temp

    data.index = range(0, len(data))    # Reassign the index
    
    return data[:24] if len(data) > 24 else data    # Return first 24 elements of data if it is greater than 24, other wise return data

In [80]:
stocks = [x for _, x in all_data.groupby('Sector')]
stocks = list(map(filter_betas, stocks))
stocks

[  Ticker   Beta                  Sector
 0   T.TO  0.722  Communication Services,
   Ticker   Beta             Sector
 0   AMZN  1.146  Consumer Cyclical,
   Ticker  Beta              Sector
 0     MO  0.67  Consumer Defensive
 1     KO  0.62  Consumer Defensive,
   Ticker   Beta              Sector
 0    USB  1.040  Financial Services
 1     BK  1.060  Financial Services
 2    AIG  1.069  Financial Services
 3  RY.TO  0.842  Financial Services
 4  TD.TO  0.822  Financial Services
 5    AXP  1.214  Financial Services
 6    BLK  1.311  Financial Services
 7    BAC  1.325  Financial Services,
   Ticker   Beta      Sector
 0    ABT  0.722  Healthcare
 1    PFE  0.615  Healthcare
 2   ABBV  0.613  Healthcare
 3    UNH  0.591  Healthcare,
   Ticker   Beta       Sector
 0    UPS  0.982  Industrials
 1    UNP  1.056  Industrials
 2    CAT  1.095  Industrials,
   Ticker   Beta      Sector
 0    TXN  0.980  Technology
 1  BB.TO  1.068  Technology
 2   AAPL  1.240  Technology
 3    ACN  1.245  

In [70]:
all_data = filter_betas(all_data)
all_data.index = all_data.index + 1

In [71]:
tsx60_info = yf.Ticker('XIU.TO')
tsx60 = tsx60_info.history(start=START_DATE, end=END_DATE, interval='1d')

sp500_info = yf.Ticker('^GSPC')
sp500 = sp500_info.history(start=START_DATE, end=END_DATE, interval='1d')
sp500 = sp500.reindex(tsx60.index, method='nearest')

market_returns = (tsx60['Close'].pct_change() + sp500['Close'].pct_change())/2
market_returns.index = market_returns.index.strftime('%Y-%m-%d')
market_returns = market_returns.dropna()
market_returns_mean = market_returns.mean()

market_returns_df = pd.DataFrame(market_returns)

In [72]:
# all_data
ticker_returns = pd.DataFrame(columns=all_data['Ticker'])
for ticker in all_data['Ticker']:
        data = yf.Ticker(ticker).history(start=START_DATE, end=END_DATE)['Close']
        ticker_returns[ticker] = data.pct_change().dropna()  # Calculate daily returns

ticker_returns.index = ticker_returns.index.strftime('%Y-%m-%d')

common_dates = ticker_returns.index.intersection(market_returns_df.index)
ticker_returns = ticker_returns.loc[common_dates]
market_returns_df = market_returns_df.loc[common_dates]

ticker_returns = ticker_returns.dropna()
market_returns_df = market_returns_df.dropna()

In [73]:
portfolio_data = []


def portfolio_generator(tickers):
    global portfolio_data
    min_result = float('inf')
    num_stocks = 0 
    final_weights = 0

    for i in range(12, 25):  
        selected_tickers = tickers[:i]  
        # print(f"Iteration {i}, Selected Tickers: {tickers[:i]}")


        def objective(weights):
            return np.std(np.dot(ticker_returns[selected_tickers].dropna().values, weights) - market_returns_df.dropna().values)
        
        # Initial weights (equal distribution) - AI
        initial_weights = [1 / len(selected_tickers)] * len(selected_tickers)
        
        # Constraints: Weights must sum to 1 - AI
        constraints = {'type': 'eq', 'fun': lambda w: sum(w) - 1}
        
        # Bounds: Weights between 0 and 1 - AI. Manually changed code provided by AI to change the constraints on the weights
        bounds = [(1/(2*i), 0.15)] * len(selected_tickers)
        
        result = minimize(objective, initial_weights, constraints=[constraints], bounds=bounds)
        # print(result.fun)

        
        if result.success:
            if result.fun < min_result:
                min_result = result.fun
                num_stocks = i
                final_weights = result.x
        
        portfolio_data.append({
            'tickers': ', '.join(selected_tickers), #AI
            'weights': final_weights,
            'STD Between Stock and Market Index': min_result,
            'num_stocks': num_stocks
        })
        
    return [pd.DataFrame(portfolio_data), min_result]
    
# all_data['Ticker']
portfolio_df = portfolio_generator(all_data['Ticker'])[0]
min_std = portfolio_generator(all_data['Ticker'])[1]

portfolio_df

,tickers,weights,STD Between Stock and Market Index,num_stocks
0,"UPS, TXN, USB, UNP, BK, BB.TO, AIG, CAT, AMZN,...","[0.04357663635612317, 0.04166666666666669, 0.0...",0.010249,12
1,"UPS, TXN, USB, UNP, BK, BB.TO, AIG, CAT, AMZN,...","[0.038461538461538464, 0.038461538461538464, 0...",0.009864,13
2,"UPS, TXN, USB, UNP, BK, BB.TO, AIG, CAT, AMZN,...","[0.03571428571428571, 0.03571428571428571, 0.0...",0.009721,14
3,"UPS, TXN, USB, UNP, BK, BB.TO, AIG, CAT, AMZN,...","[0.03333333333333337, 0.033333333333333354, 0....",0.009295,15
4,"UPS, TXN, USB, UNP, BK, BB.TO, AIG, CAT, AMZN,...","[0.03125, 0.03125, 0.03125, 0.03125, 0.0318445...",0.008976,16
5,"UPS, TXN, USB, UNP, BK, BB.TO, AIG, CAT, AMZN,...","[0.03125, 0.03125, 0.03125, 0.03125, 0.0318445...",0.008976,16
6,"UPS, TXN, USB, UNP, BK, BB.TO, AIG, CAT, AMZN,...","[0.03125, 0.03125, 0.03125, 0.03125, 0.0318445...",0.008976,16
7,"UPS, TXN, USB, UNP, BK, BB.TO, AIG, CAT, AMZN,...","[0.03125, 0.03125, 0.03125, 0.03125, 0.0318445...",0.008976,16
8,"UPS, TXN, USB, UNP, BK, BB.TO, AIG, CAT, AMZN,...","[0.025, 0.025, 0.025, 0.02500000000000001, 0.0...",0.008827,20
9,"UPS, TXN, USB, UNP, BK, BB.TO, AIG, CAT, AMZN,...","[0.02380952380952381, 0.023809523809523815, 0....",0.008604,21


In [64]:
portfolio_data

[{'tickers': 'UPS, TXN, USB, UNP, BK, BB.TO, AIG, CAT, AMZN, RY.TO, TD.TO, AXP',
  'weights': array([1.23714841e-02, 0.00000000e+00, 0.00000000e+00, 1.28870540e-01,
         1.26298311e-01, 8.67361738e-18, 9.54692023e-02, 0.00000000e+00,
         5.84525623e-02, 3.07877306e-01, 2.70660594e-01, 0.00000000e+00]),
  'STD Between Stock and Market Index': 0.00947722176421311,
  'num_stocks': 12},
 {'tickers': 'UPS, TXN, USB, UNP, BK, BB.TO, AIG, CAT, AMZN, RY.TO, TD.TO, AXP, AAPL',
  'weights': array([3.34419461e-03, 1.73472348e-18, 1.38777878e-17, 1.17723633e-01,
         1.21826788e-01, 1.27979757e-18, 9.46546992e-02, 2.51534904e-17,
         4.51156357e-03, 2.41117191e-01, 2.26075149e-01, 0.00000000e+00,
         1.90746782e-01]),
  'STD Between Stock and Market Index': 0.0091733299979708,
  'num_stocks': 13},
 {'tickers': 'UPS, TXN, USB, UNP, BK, BB.TO, AIG, CAT, AMZN, RY.TO, TD.TO, AXP, AAPL, ACN',
  'weights': array([9.45416575e-03, 3.03576608e-18, 6.28837260e-18, 6.76458235e-02,
    

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.